In [ ]:
#Installing Nesscary libraries
%pip install opencv-python
%pip install mediapipe
%pip install cvzone

In [1]:
"""
FINDING THE FOCAL LENGTH OF YOUR CAMERA
NOTE:
    Before running this part of the program ensure to sit correctly 57cms away from your camera.
"""
#Importing Libraries
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector

#defining custom method for calculating distance
def FocalLength(measured_distance, real_width, width_in_rf_image):
    focal_length = (width_in_rf_image* measured_distance)/ real_width
    return focal_length

#intailizing to capture framesfor processing.
cap = cv2.VideoCapture(0)

#creating instance of class 'FaceMeshDetector' with max face count of 1
detector = FaceMeshDetector(maxFaces=1)

#Looping to process captured frames continuesly 
while True:
    
    #cap.read() reads the frame and saves it in img.Success is boolean stating if img is true or not
    success, img = cap.read()
    
    #Calling findFaceMesh method and passing img and store the mesh info in "faces"."draw=False" specifes to not display the mesh.
    img, faces = detector.findFaceMesh(img, draw=False)
    
    # if Face detected pass
    if faces:
        
        #Mark the first face of the list "Faces"
        face = faces[0]
        
        #Marking eye points for finding pixel distance
        pointLeft = face[145]
        pointRight = face[374]
        
        '''
        The calculated distance is assigned to the variable w, and the underscore _ is used as 
        a placeholder for the second return value, which is not being used or stored in any variable.
        By assigning the calculated distance to the variable w, you can use this value further 
        in your code for any required computations or comparisons.
        '''
        w, _ = detector.findDistance(pointLeft, pointRight)
        #Declaring Known instances
        W = 6.3#Distance between eyes in cms
        d = 57#Standard distnce between camera and user in cms
        #Finding the Focal Length
        Foclen = FocalLength(d, W, w)
        #Printing the calculated focal length
        cv2.putText(img, f'FocalLength: {int(Foclen)}cm',(face[10][0] - 100, face[10][1] - 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Image", img)
    #exit loop command using 'q' key(also case sensitive)
    if cv2.waitKey(1)==ord("q"):
            break 
# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

In [3]:
'''
FINDING THE DISTANCE OF FACE FROM CAMERA
NOTE:
    Entering the correct focal length will increase the accuracy of the score.
'''

#impoerting the libraries
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.FaceDetectionModule import FaceDetector
#Defining custom method for calculating Distance
def Distance_finder (Focal_Length, real_face_width, face_width_in_frame):
    distance = (real_face_width * Focal_Length)/face_width_in_frame
    return distance

#intailizing to capture framesfor processing.
cap = cv2.VideoCapture(0)

#creating instance of class 'FaceMeshDetector' with max face count of 1
detector = FaceMeshDetector(maxFaces=1)

#creating instance of class 'FaceDetector' from cvzone
face_detector = FaceDetector()

#Looping to process captured frames continuesly 
while True:
    #cap.read() reads the frame and saves it in img.Success is boolean stating if img is true or not
    success, img = cap.read()
    
    '''
    Calling findFaceMesh method and passing img and store the mesh info in "faces"."draw=False" 
    specifes to not display the mesh.
    '''    
    img, faces = detector.findFaceMesh(img, draw=False)
    '''
    Calling 'findFaces'method and and marking the face found in img."draw=True" marks the boundary
    around the face.
    '''
    frame,bboxs = face_detector.findFaces(img, draw=False)
           
    # if Face detected pass
    if faces:
        
        #Mark the first face of the list "Faces"
        face = faces[0]
        
        #Marking eye points for finding pixel distance
        pointLeft = face[145]
        pointRight = face[374]
        
        '''
        The calculated distance is assigned to the variable w, and the underscore _ is used as 
        a placeholder for the second return value, which is not being used or stored in any variable.
        By assigning the calculated distance to the variable w, you can use this value further 
        in your code for any required computations or comparisons.
        '''
        w, _ = detector.findDistance(pointLeft, pointRight)
        
        #Declaring Known instances
        #Real distance between the eyes in cms
        Real_Dist = 6.3
        
        #focal length of the camera found
        Focal_Length = 615
        
        # Finding distance
        d = Distance_finder (Focal_Length, Real_Dist, w)
        
        #Fetching the BBOX coordinates from the Facedetector's outputs 
        bbox = bboxs[0]["bbox"]
        
        #printing Boundary
        img = cv2.rectangle(frame, bbox, (255, 0, 255), 2)
        
        #printing the dynamic distance
        cv2.putText(img, f'Depth: {int(d)}cm',
                           (face[10][0] - 100, face[10][1] - 50),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Image", img)
    #exit command using 'q' key(case sensitive)
    if cv2.waitKey(1)==ord("q"):
            break 
# Release the video capture and close windowsq
cap.release()
cv2.destroyAllWindows()